In [20]:
import pandas as pd

df = pd.read_csv("./dataset.csv")
df.columns = df.columns.str.lower()


In [21]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   area             900 non-null    int64  
 1   majoraxislength  900 non-null    float64
 2   minoraxislength  900 non-null    float64
 3   eccentricity     900 non-null    float64
 4   convexarea       900 non-null    int64  
 5   extent           900 non-null    float64
 6   perimeter        900 non-null    float64
 7   class            900 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 56.4+ KB


In [22]:
df['class'] = pd.factorize(df['class'])[0]
df


,area,majoraxislength,minoraxislength,eccentricity,convexarea,extent,perimeter,class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,0
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,0
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,0
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,0
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,0
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,1
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,1
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,1
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,1


In [23]:
df.isnull().sum()


area               0
majoraxislength    0
minoraxislength    0
eccentricity       0
convexarea         0
extent             0
perimeter          0
class              0
dtype: int64